In [2]:
import Load_CIFAR10
import numpy as  np
from sklearn import preprocessing

In [3]:
from imp import reload 
reload(Load_CIFAR10) 

<module 'Load_CIFAR10' from 'F:\\JupyterNoteBookWorkenEnviroment\\MachineLearning\\NetWork\\Load_CIFAR10.py'>

In [3]:
trainX,trainY,testX,testY = Load_CIFAR10.load_Data_Lables()
#拉成一维向量
trainX = trainX.reshape(trainX.shape[0],-1)
testX = testX.reshape(testX.shape[0],-1)
#标准化
scaler = preprocessing.StandardScaler().fit(trainX)
trainX = scaler.transform(trainX)
testX = scaler.transform(testX)
#给每个数据都要多加一维，作为偏置位
train_bias = np.ones([trainX.shape[0],1],float)
test_bias = np.ones([testX.shape[0],1],float)
trainX = np.c_[trainX,train_bias]
testX = np.c_[testX,test_bias]

D:\Anaconda\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype uint8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [58]:
print(np.max(trainY))

9


In [16]:
class NeuralNetwork:
    def __init__(self,layers,alpha):
        '''初始函数
        layers:tuple类型,每一个元素表示该层的单元个数
        alpha:学习率
        '''
        self.layers = layers
        self.W = []
        self.dropOut = [None for i in range(len(layers)-1)]
         
        self.alpha = alpha
        self.out = [None for i in range(len(layers)-1)]
        self.result = [None for i in range(len(layers))]#每一层运算后的结果
        
        for i in range(len(layers)-2):
            #输入层和输出层不需要权重
            #要考虑偏置值
            w = np.random.randn(layers[i]+1,layers[i+1]+1)
            self.W.append(w)
        
        #最后一层输出层不需要加偏置项
        w = np.random.randn(layers[-2]+1,layers[-1])
        self.W.append(w)
        
        #转换一下类型
        self.W = np.array(self.W)
        self.result = np.array(self.result)

        
    def Relu(self,value):
        '''激活函数采用整流线性单元，但是为了避免出现梯度饱和，采用渗漏型整流线性单元
        value:上一层的输出入与隐藏层的权重乘积的结果
        '''
        v = []
        for i in value:
            d = [j if j>=0 else j*0.01 for j in i]
            v.append(d)
        return np.array(v)

    def softMax(self,value):
        x = np.exp(value)
        return np.array([ i/np.sum(i) for i in x])
    
    def fit(self,trainX,trainY,batch_size,reg,num):
        num_train = trainX.shape[0]
        for i in range(num):
            #进行小批量训练，随机抽取一批数量为batch_size的数据集进行训练
            batch_index = np.random.choice(num_train,batch_size)
            batch_X = trainX[batch_index,:]
            batch_Y = trainY[batch_index]
            self.train(batch_X,batch_Y,0.2)
            print('loss',self.calculateLoss(trainX,trainY))
        
    def getdropOutVector(self,size):
        '''获取一次用于dropOut的向量Load_CIFAR10.py
        size:表示几行几列
        '''
        return np.random.randint(0,2,size)
    
    def calculateLoss(self,trainX,trainY):
        out = trainX
        #计算隐藏层
        for i in range(1,len(self.layers)-1):
            #上一层的输出与当前层的权重相乘
            net = out.dot(self.W[i-1])
            
            scaler = preprocessing.StandardScaler().fit(net)
            net = scaler.transform(net)
            #生成当前层dropout向量
            #保存到self.dropOut中去，用于反向传播
            #再把Dropout向量乘上当前的输出
#             dropOutVec = self.getdropOutVector([1,net.shape[1]])
#             net = net*dropOutVec
            
            #经过激活函数
            out = self.Relu(net)
        
        #计算输出层
        net = out.dot(self.W[-1])#因为数据没有归一化或者标准化化？导致数据过大，爆了。。
        out = self.softMax(net)
        
        loss = sum([np.log(out[i][trainY[i]]) for i in range(out.shape[0])])
        regular_term = self.W**2
        regular_term = [np.sum(i) for i in regular_term]
        regular_term = np.sum(regular_term)
        
        return int(loss)
    
    def train(self,batch_X,batch_Y,reg):
        self.result[0] = batch_X
        
        #计算隐藏层
        for i in range(1,len(self.layers)-1):
            
            #上一层的输出与当前层的权重相乘
            net = self.result[i-1].dot(self.W[i-1])
            
            scaler = preprocessing.StandardScaler().fit(net)
            net = scaler.transform(net)
            
            #生成当前层dropout向量
            #保存到self.dropOut中去，用于反向传播
            #再把Dropout向量乘上当前的输出
            dropOutVec = self.getdropOutVector([1,net.shape[1]])
            self.dropOut[i-1] = dropOutVec
            net = net*dropOutVec
            
            #经过激活函数
            out = self.Relu(net)
            self.out[i-1] = out
            self.result[i] = out
        
        #计算输出层
        net = self.result[-2].dot(self.W[-1])#因为数据没有归一化或者标准化化？导致数据过大，爆了。。
        out = self.softMax(net)
        self.result[-1] = out
        
        #反向传播
        #先是输出层的反向传播梯度更新
        df_dout = np.array([ [self.result[-1][i][batch_Y[i]]] for i in range(self.result[-1].shape[0])])
        df_dout = 1.0/df_dout
        
        dout_dnet = []
        for i in range(batch_X.shape[0]):
            _dout_dnet = [(np.sum(self.result[-1][i])-self.result[-1][i][j])*self.result[-1][i][j]/(np.sum(self.result[-1][i])**2) for j in range(self.layers[-1])]
            dout_dnet.append(_dout_dnet)
        dout_dnet = np.array(dout_dnet)

        D = [None for i in range(len(self.W))]#保存 df/dout *dout/dnet
        D[-1] = df_dout*dout_dnet
        
        #接着是隐藏层的
        for i in range(len(self.W)-2,-1,-1):
            dout_dnet = []
            for j in self.out[i]:
                _dout_dnet = [1 if m>=0 else 0.01 for m in j]
                dout_dnet.append(_dout_dnet)
                
            dout_dnet = np.array(dout_dnet)
    
            D[i] = D[i+1].dot(self.W[i+1].T)*dout_dnet#注意点乘还是*乘
        
        #为输入出增加一层全都是1的dropout层，输入层是不需要dropout的，为了公式一致所以加上一层都是为1的dropout层
        last_dropoutVector = np.array([1.0 for i in range(self.W[-1].shape[-1])])
        self.dropOut[-1] = last_dropoutVector
          
        #然后是梯度更新
        for i in range(len(self.W)):
            print(self.result[i].shape)
            print(D[i].shape)
            self.W[i] = self.W[i] - self.alpha*(self.W[i]*reg+self.result[i].T.dot(D[i]*self.dropOut[i]))#注意点乘还是*乘
            print('dw',(self.alpha*(self.W[i]*reg+self.result[i].T.dot(D[i]*self.dropOut[i]))).shape)
        

In [17]:
bp_net = NeuralNetwork([trainX.shape[1]-1,64,32,16,10],2e-6)
bp_net.fit(trainX=trainX[:5000],trainY=trainY[:5000],batch_size=10,num=1,reg=0.9)

(10, 3073)
(10, 65)
dw (3073, 65)
(10, 65)
(10, 33)
dw (65, 33)
(10, 33)
(10, 17)
dw (33, 17)
(10, 17)
(10, 10)
dw (17, 10)
loss -22181


In [ ]:
# a = [[1,2,3],[2,3,4]]
# b = [j if j<2 else 0.01*j for i in a for j in i ]
# print(b)
a = np.array([[1,2,3,4],[1,2,3]])
print(np.sum(a**2))

In [39]:

# c = [1,2,3]
# c = np.exp(c)
# print(c)
# _sum = np.sum(c)
# print(c/_sum)
# c = [[1,2,3],[2,3,4]]
# print(np.sum(c,1))
# help(np.full)

# d = [ (1.0*3/(2*2)) for i in range(2) for i in range(2)]
# e = np.atleast_2d(d)
# d = [ (1.0*3/(2*2)) for i in range(2) for i in range(2)]
# e = np.insert(e,0,d,0)
# print(e)

# e = [[1,2],[2,3],[3,4]]
# e = np.array(e)
# d = [[0,0],[-1,-2],[-2,-3]]
# d = np.array(d)

# print(e*d)
d = np.array([[1,2],[1,2]])
c = np.ones([d.shape[0],1],float)
e = np.c_[d,c]
print(e)


[[1. 2. 1.]
 [1. 2. 1.]]


In [91]:
a = [1,2,3,4]
b = [1,2,3,4]
for i,j in a,b:
    print(i,j)

ValueError: too many values to unpack (expected 2)